In [ ]:
import pandas as pd

sheet_name = "Opera San Jose"
df = pd.read_excel('Repertoire.xlsx',sheet_name=sheet_name)
df.head()

In [ ]:
df = df.loc[(df['Repertoire'].isnull()==True)]
df.rename(columns={"Event Name": "EventName"}, inplace=True)
df[['EventName','Repertoire']].to_csv("output.csv", index=False)
df.head()

In [ ]:
sample_std_df = pd.read_csv('output.csv')
variables = dict(zip(sample_std_df.EventName,sample_std_df.Repertoire))
# variables = sample_std_df.to_dict(orient="series")
# variables_str = '\n'.join(str(row) for row in variables)
variables

In [8]:
import os

from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import SystemMessage
from langchain.output_parsers import PydanticOutputParser

In [26]:
openai_devapi = os.getenv("openaidev_api")
# print(openai_devapi)

In [ ]:
repertoire_generation_prompt = """
"""

In [122]:
chat = ChatOpenAI(temperature=0.1, model_name="gpt-4o", openai_api_key=openai_devapi)

prompt_template = PromptTemplate(template=repertoire_generation_prompt, input_variables=variables)


system_message = SystemMessagePromptTemplate(prompt=prompt_template)
chat_message = system_message.format_messages(variables=variables)

response = chat.invoke(chat_message)

In [ ]:
print(chat_message)

In [ ]:
# response_cincinatti = response.content
print(response.content)

In [ ]:
# with open("response_sanjose.txt","a") as f:
#     f.write(response.content)

In [141]:
final_response = response.content

if "```json" in final_response:
    _, final_response = final_response.split("```json")
if "```" in final_response:
    final_response, _ = final_response.split("```")

In [ ]:
print(final_response)

In [ ]:
import json

# df = pd.read_excel("Repertoire.xlsx")

final_json = json.loads(final_response)
final_df = pd.DataFrame(final_json, index=["Event Name"]).transpose().reset_index()
final_df = final_df.rename(columns={"index": "Event Name", "Event Name": "Repertoire"})
# final_df.to_csv("output.csv", index=False)
final_file = pd.merge(final_df, df, how="outer").drop_duplicates(subset=['Event Name'])
final_file
# final_file.replace('New', 'New Work', inplace=True)
# final_file.replace('Top20', 'Top 20', inplace=True)
# final_file.to_excel(f"{sheet_name}.xlsx", index=False)